In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6249400819943235782, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 523992018666209774
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time
import math
import os
import pandas as pd
from keras.callbacks import EarlyStopping
import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize
from keras.utils.np_utils import to_categorical
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [5]:
batch_size = 32
img_height, img_width = 128, 128
input_shape = (img_height, img_width, 3)
epochs = 15

In [6]:
train_dir = 'data/train/'
test_dir = 'data/test'

In [7]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split= 0.3,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 34909 images belonging to 6 classes.
Found 14958 images belonging to 6 classes.
Found 13578 images belonging to 6 classes.


In [8]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 34909
nb_validation_samples: 14958
nb_test_samples: 13578

predict_size_train: 1091
predict_size_validation: 468
predict_size_test: 425

 num_classes: 6


In [9]:
# vgg19_weights ="../input/full-keras-pretrained-no-top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_weights ="../input/full-keras-pretrained-no-top//inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
# vgg16_weights ="../input/full-keras-pretrained-no-top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet201_weights ="../input/full-keras-pretrained-no-top/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet121_weights ="../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
# resenet50_weights ="../input/full-keras-pretrained-no-top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_resnet_v2_weights ="../input/full-keras-pretrained-no-top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
# nasnet_weights ="../input/full-keras-pretrained-no-top/nasnet_large_no_top.h5"
# nasnet_mobile_weights ="../input/full-keras-pretrained-no-top/nasnet_mobile_no_top.h5"
# mobilenet_weights ="../input/full-keras-pretrained-no-top/mobilenet_1_0_224_tf_no_top.h5"

In [10]:
# from keras.applications.vgg19 import VGG19
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.vgg16 import VGG16
# from keras.applications.xception import Xception, preprocess_input
# from keras.applications import DenseNet201
# from keras.applications import DenseNet121
# from keras.applications import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from keras.applications import NASNetLarge, NASNetMobile
# from keras.applications import MobileNet

bottleneck_final_model=InceptionResNetV2(weights="imagenet", include_top=False, pooling = "avg")

In [11]:
# os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "InceptionResNetV2_descriptors"

In [12]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train, max_q_size=1, pickle_safe=False)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [13]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [14]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [15]:

import tensorflow as tf
from keras.backend.tensorflow_backend import get_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import set_session

def reset_keras_tf_session():
    """
    this function clears the gpu memory and set the 
    tf session to not use the whole gpu
    """
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

#     config = tf.ConfigProto()
#     config.gpu_options.allow_growth = True
#     set_session(tf.Session(config=config))

reset_keras_tf_session()

In [16]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [17]:
space = {
         'lr': hp.choice('lr',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.9]),
         'beta_2':hp.choice('beta_2',[0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.999,0.99,0.995]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [18]:
from sklearn.metrics import accuracy_score
def f_nn(params):   
    print ('Parameters testing: ', params)
  
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0)
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = Sequential()
    # model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.7))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(validation_data, validation_labels),
                        verbose= 2,
                        callbacks=get_callbacks(params))

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]
#     cm = confusion_matrix(y_pred=predictions, y_true=y_true)

    print('Test Accuracy: {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [19]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [20]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals = 300, trials=trials)
print(best)

Parameters testing:  {'beta_1': 0.7, 'lr': 0.01, 'beta_2': 0.999}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 4s - loss: 1.1534 - acc: 0.6343 - val_loss: 0.9620 - val_acc: 0.6090
Epoch 2/15
 - 4s - loss: 1.0454 - acc: 0.6431 - val_loss: 1.0714 - val_acc: 0.5907
Epoch 3/15
 - 4s - loss: 1.0353 - acc: 0.6446 - val_loss: 1.0079 - val_acc: 0.6308
Epoch 4/15
 - 4s - loss: 1.0518 - acc: 0.6450 - val_loss: 0.9782 - val_acc: 0.6322
Epoch 5/15
 - 4s - loss: 1.1069 - acc: 0.6375 - val_loss: 1.0862 - val_acc: 0.6100
Epoch 6/15
 - 4s - loss: 1.0546 - acc: 0.6503 - val_loss: 1.0229 - val_acc: 0.6423
Epoch 7/15
 - 4s - loss: 1.0383 - acc: 0.6553 - val_loss: 0.9752 - val_acc: 0.6463
Epoch 8/15
 - 4s - loss: 1.0283 - acc: 0.6581 - val_loss: 1.0415 - val_acc: 0.6449
Epoch 9/15
 - 4s - loss: 1.0286 - acc: 0.6662 - val_loss: 1.0625 - val_acc: 0.6432
Epoch 10/15
 - 4s - loss: 1.0314 - acc: 0.6580 - val_loss: 1.0432 - val_acc: 0.6366
Epoch 00010: early stopping
14958/14958 [============

14958/14958 [==============================] - 1s 36us/step
Validation Accuracy: 32.4308%
Validation Loss: 1.6521170814975505
Test Accuracy: 0.45669465311533364
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.9, 'lr': 0.001, 'beta_2': 0.6}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 4s - loss: 1.3899 - acc: 0.6395 - val_loss: 1.1876 - val_acc: 0.6423
Epoch 2/15
 - 4s - loss: 3.6259 - acc: 0.5237 - val_loss: 7.2266 - val_acc: 0.3742
Epoch 3/15
 - 4s - loss: 1.7195 - acc: 0.5607 - val_loss: 1.2188 - val_acc: 0.6078
Epoch 4/15
 - 4s - loss: 1.1030 - acc: 0.5951 - val_loss: 0.9866 - val_acc: 0.6174
Epoch 00004: early stopping
14958/14958 [==============================] - 1s 36us/step
Validation Accuracy: 61.7395%
Validation Loss: 0.9866229425067251
Test Accuracy: 0.7366327883340699
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_

Parameters testing:  {'beta_1': 0.1, 'lr': 1e-05, 'beta_2': 0.8}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.3562 - acc: 0.5221 - val_loss: 1.2860 - val_acc: 0.5097
Epoch 2/15
 - 4s - loss: 0.9393 - acc: 0.6835 - val_loss: 1.1845 - val_acc: 0.5421
Epoch 3/15
 - 4s - loss: 0.8334 - acc: 0.7196 - val_loss: 1.1295 - val_acc: 0.5676
Epoch 4/15
 - 4s - loss: 0.7685 - acc: 0.7381 - val_loss: 1.0873 - val_acc: 0.5838
Epoch 5/15
 - 4s - loss: 0.7301 - acc: 0.7532 - val_loss: 1.0596 - val_acc: 0.5979
Epoch 6/15
 - 4s - loss: 0.6948 - acc: 0.7641 - val_loss: 1.0335 - val_acc: 0.6084
Epoch 7/15
 - 4s - loss: 0.6692 - acc: 0.7739 - val_loss: 1.0141 - val_acc: 0.6189
Epoch 8/15
 - 4s - loss: 0.6460 - acc: 0.7818 - val_loss: 0.9981 - val_acc: 0.6254
Epoch 9/15
 - 4s - loss: 0.6323 - acc: 0.7870 - val_loss: 0.9852 - val_acc: 0.6296
Epoch 10/15
 - 4s - loss: 0.6183 - acc: 0.7906 - val_loss: 0.9723 - val_acc: 0.6347
Epoch 11/15
 - 4s - loss: 0.6015 - acc: 0.7960 - val_l

Test Accuracy: 0.6644572101929592
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.4, 'lr': 0.001, 'beta_2': 0.4}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 0.7986 - acc: 0.7703 - val_loss: 1.1324 - val_acc: 0.6911
Epoch 2/15
 - 4s - loss: 0.7903 - acc: 0.8125 - val_loss: 1.4356 - val_acc: 0.6873
Epoch 3/15
 - 4s - loss: 0.8694 - acc: 0.8219 - val_loss: 1.7126 - val_acc: 0.6853
Epoch 4/15
 - 4s - loss: 0.9131 - acc: 0.8261 - val_loss: 1.6136 - val_acc: 0.7083
Epoch 5/15
 - 4s - loss: 0.9851 - acc: 0.8298 - val_loss: 1.8022 - val_acc: 0.7006
Epoch 6/15
 - 4s - loss: 1.0570 - acc: 0.8318 - val_loss: 2.0021 - val_acc: 0.7028
Epoch 7/15
 - 4s - loss: 1.0993 - acc: 0.8331 - val_loss: 1.9731 - val_acc: 0.6982
Epoch 00007: early stopping
14958/14958 [==============================] - 1s 38us/step
Validation Accuracy: 69.8222%
V

Epoch 14/15
 - 4s - loss: 0.5531 - acc: 0.8107 - val_loss: 0.9363 - val_acc: 0.6530
Epoch 15/15
 - 4s - loss: 0.5420 - acc: 0.8147 - val_loss: 0.9250 - val_acc: 0.6582
14958/14958 [==============================] - 1s 39us/step
Validation Accuracy: 65.8243%
Validation Loss: 0.92497113420075
Test Accuracy: 0.7793489468257475
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.3, 'lr': 1e-05, 'beta_2': 0.995}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.4374 - acc: 0.4961 - val_loss: 1.3340 - val_acc: 0.4970
Epoch 2/15
 - 4s - loss: 0.9467 - acc: 0.6832 - val_loss: 1.1998 - val_acc: 0.5384
Epoch 3/15
 - 4s - loss: 0.8232 - acc: 0.7219 - val_loss: 1.1154 - val_acc: 0.5674
Epoch 4/15
 - 4s - loss: 0.7514 - acc: 0.7434 - val_loss: 1.0727 - val_acc: 0.5859
Epoch 5/15
 - 4s - loss: 0.7050 - acc: 0.7576 - val_loss: 1.0390 - val_acc

...

Epoch 10/15
 - 8s - loss: 1.0384 - acc: 0.6356 - val_loss: 1.3009 - val_acc: 0.5006
Epoch 11/15
 - 9s - loss: 1.0118 - acc: 0.6475 - val_loss: 1.2851 - val_acc: 0.5062
Epoch 12/15
 - 9s - loss: 0.9879 - acc: 0.6591 - val_loss: 1.2680 - val_acc: 0.5103
Epoch 13/15
 - 9s - loss: 0.9692 - acc: 0.6660 - val_loss: 1.2537 - val_acc: 0.5146
Epoch 14/15
 - 9s - loss: 0.9448 - acc: 0.6762 - val_loss: 1.2413 - val_acc: 0.5182
Epoch 15/15
 - 9s - loss: 0.9301 - acc: 0.6852 - val_loss: 1.2300 - val_acc: 0.5228
14958/14958 [==============================] - 2s 146us/step
Validation Accuracy: 52.2797%
Validation Loss: 1.2299989321633829
Test Accuracy: 0.6814700250405067
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.7, 'lr': 1e-05, 'beta_2': 0.995}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 20s - loss: 1.4860 - acc: 0.4723 - val_loss: 1.3162 -

Epoch 4/15
 - 9s - loss: 0.4717 - acc: 0.8361 - val_loss: 0.8700 - val_acc: 0.6774
Epoch 5/15
 - 9s - loss: 0.4390 - acc: 0.8428 - val_loss: 0.8292 - val_acc: 0.6872
Epoch 6/15
 - 8s - loss: 0.4221 - acc: 0.8512 - val_loss: 0.8261 - val_acc: 0.6917
Epoch 7/15
 - 7s - loss: 0.4045 - acc: 0.8564 - val_loss: 0.8249 - val_acc: 0.6972
Epoch 8/15
 - 7s - loss: 0.3951 - acc: 0.8594 - val_loss: 0.8435 - val_acc: 0.6929
Epoch 9/15
 - 7s - loss: 0.3829 - acc: 0.8632 - val_loss: 0.8494 - val_acc: 0.6976
Epoch 10/15
 - 7s - loss: 0.3713 - acc: 0.8682 - val_loss: 0.8166 - val_acc: 0.7023
Epoch 11/15
 - 7s - loss: 0.3610 - acc: 0.8715 - val_loss: 0.8127 - val_acc: 0.7036
Epoch 12/15
 - 8s - loss: 0.3563 - acc: 0.8721 - val_loss: 0.7952 - val_acc: 0.7081
Epoch 13/15
 - 9s - loss: 0.3498 - acc: 0.8756 - val_loss: 0.8426 - val_acc: 0.7050
Epoch 14/15
 - 8s - loss: 0.3412 - acc: 0.8776 - val_loss: 0.8577 - val_acc: 0.7002
Epoch 15/15
 - 9s - loss: 0.3346 - acc: 0.8814 - val_loss: 0.8172 - val_acc: 0.709

Epoch 7/15
 - 9s - loss: 0.6480 - acc: 0.7772 - val_loss: 0.9976 - val_acc: 0.6195
Epoch 8/15
 - 9s - loss: 0.6223 - acc: 0.7862 - val_loss: 0.9761 - val_acc: 0.6298
Epoch 9/15
 - 9s - loss: 0.6047 - acc: 0.7930 - val_loss: 0.9580 - val_acc: 0.6357
Epoch 10/15
 - 9s - loss: 0.5885 - acc: 0.7994 - val_loss: 0.9488 - val_acc: 0.6381
Epoch 11/15
 - 9s - loss: 0.5740 - acc: 0.8035 - val_loss: 0.9292 - val_acc: 0.6457
Epoch 12/15
 - 9s - loss: 0.5567 - acc: 0.8082 - val_loss: 0.9322 - val_acc: 0.6453
Epoch 13/15
 - 9s - loss: 0.5491 - acc: 0.8115 - val_loss: 0.9182 - val_acc: 0.6502
Epoch 14/15
 - 9s - loss: 0.5381 - acc: 0.8145 - val_loss: 0.9056 - val_acc: 0.6552
Epoch 15/15
 - 9s - loss: 0.5290 - acc: 0.8170 - val_loss: 0.8998 - val_acc: 0.6558
14958/14958 [==============================] - 3s 180us/step
Validation Accuracy: 65.5769%
Validation Loss: 0.8998397330594486
Test Accuracy: 0.784725290911769
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*

 - 9s - loss: 0.3719 - acc: 0.8685 - val_loss: 0.8385 - val_acc: 0.7038
Epoch 00010: early stopping
14958/14958 [==============================] - 3s 172us/step
Validation Accuracy: 70.3771%
Validation Loss: 0.8384601116725341
Test Accuracy: 0.8264840182648402
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.8, 'lr': 0.1, 'beta_2': 0.999}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 21s - loss: 12.1329 - acc: 0.2464 - val_loss: 12.7475 - val_acc: 0.2091
Epoch 2/15
 - 9s - loss: 12.7157 - acc: 0.2111 - val_loss: 12.7475 - val_acc: 0.2091
Epoch 3/15
 - 10s - loss: 12.5951 - acc: 0.2185 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 4/15
 - 9s - loss: 12.5274 - acc: 0.2228 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 5/15
 - 9s - loss: 12.3604 - acc: 0.2331 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 6/15
 - 9s - loss: 12.3089 - acc: 0.2363

Epoch 4/15
 - 10s - loss: 0.7352 - acc: 0.7468 - val_loss: 1.0522 - val_acc: 0.5945
Epoch 5/15
 - 10s - loss: 0.6850 - acc: 0.7639 - val_loss: 1.0156 - val_acc: 0.6084
Epoch 6/15
 - 9s - loss: 0.6533 - acc: 0.7772 - val_loss: 0.9994 - val_acc: 0.6176
Epoch 7/15
 - 10s - loss: 0.6287 - acc: 0.7840 - val_loss: 0.9755 - val_acc: 0.6278
Epoch 8/15
 - 9s - loss: 0.6056 - acc: 0.7896 - val_loss: 0.9571 - val_acc: 0.6360
Epoch 9/15
 - 9s - loss: 0.5886 - acc: 0.7979 - val_loss: 0.9484 - val_acc: 0.6403
Epoch 10/15
 - 9s - loss: 0.5718 - acc: 0.8018 - val_loss: 0.9411 - val_acc: 0.6422
Epoch 11/15
 - 9s - loss: 0.5582 - acc: 0.8085 - val_loss: 0.9188 - val_acc: 0.6496
Epoch 12/15
 - 9s - loss: 0.5511 - acc: 0.8112 - val_loss: 0.9249 - val_acc: 0.6494
Epoch 13/15
 - 9s - loss: 0.5399 - acc: 0.8149 - val_loss: 0.9135 - val_acc: 0.6538
Epoch 14/15
 - 9s - loss: 0.5263 - acc: 0.8178 - val_loss: 0.9016 - val_acc: 0.6584
Epoch 15/15
 - 9s - loss: 0.5208 - acc: 0.8214 - val_loss: 0.9032 - val_acc: 0.